In [47]:
#This imports the pandas, numpy libraries and datetime to timestamp the output file
import pandas as pd
import numpy as np
from datetime import datetime

In [48]:
np.random.seed(42)

In [49]:
num_patients = 1000

In [50]:
#PART I: SYNTHETIC DATA CREATION
data = {'Patient_ID': [], 'Sex': [], 'Condition_1': [], 'Diagnosis_Date_1': [],
        'Condition_2': [], 'Diagnosis_Date_2': [], 'Condition_3': [], 'Diagnosis_Date_3': []}

In [51]:
medical_conditions = ['Diabetes', 'Hypertension', 'Asthma', 'Obesity', 'Arthritis', 'Depression', 'Migraine', 'Allergies', 'Osteoporosis', 'Anemia']

In [52]:
for patient_id in range(1, num_patients + 1):
    # Add patient ID and gender
    data['Patient_ID'].append(patient_id)
    data['Sex'].append(np.random.choice(['Male', 'Female']))

    # This generates a random number of medical conditions (at least one)
    num_conditions = np.random.randint(1, 4)
    condition_dates = [pd.Timestamp(np.random.randint(2000, 2023), np.random.randint(1, 13), np.random.randint(1, 29)) for _ in range(num_conditions)]

    # This adds a medical condition and diagnosis date
    for i in range(3):
        if i < num_conditions:
            data[f'Condition_{i+1}'].append(np.random.choice(medical_conditions))
            data[f'Diagnosis_Date_{i+1}'].append(condition_dates[i])
        else:
            data[f'Condition_{i+1}'].append(np.nan)
            data[f'Diagnosis_Date_{i+1}'].append(pd.NaT)

In [53]:
synthetic_health_data = pd.DataFrame(data)

In [54]:
synthetic_health_data

,Patient_ID,Sex,Condition_1,Diagnosis_Date_1,Condition_2,Diagnosis_Date_2,Condition_3,Diagnosis_Date_3
0,1,Male,Arthritis,2014-11-08,NaN,NaT,NaN,NaT
1,2,Male,Obesity,2018-07-11,Allergies,2010-08-21,NaN,NaT
2,3,Female,Anemia,2021-05-02,Depression,2011-06-02,Osteoporosis,2020-01-12
3,4,Male,Asthma,2009-12-28,Arthritis,2015-03-12,Asthma,2022-04-25
4,5,Male,Obesity,2008-07-18,NaN,NaT,NaN,NaT
...,...,...,...,...,...,...,...,...
995,996,Female,Asthma,2013-08-03,NaN,NaT,NaN,NaT
996,997,Male,Diabetes,2002-12-28,NaN,NaT,NaN,NaT
997,998,Female,Obesity,2020-09-23,NaN,NaT,NaN,NaT
998,999,Female,Migraine,2001-10-10,Migraine,2010-12-06,Anemia,2003-08-12


In [55]:
#Part II: CLEAN AND PIVOT.
synthetic_health_data=synthetic_health_data.rename(columns={'Sex':'Sex_1'})

#This melts the wide format to long format on the column name, separates them on '_' and stores them as numbers.
melted_df = pd.wide_to_long(synthetic_health_data,
                            stubnames=['Sex', 'Condition', 'Diagnosis_Date'],
                            i='Patient_ID',
                            j='Num',
                            sep='_')

In [56]:
melted_df = melted_df.reset_index()

In [57]:
#This prevents a condition from being double counted if a patient had it more than once.
melted_df = melted_df.drop_duplicates(subset=['Patient_ID', 'Condition'])

In [58]:
#This creates the pivot table, with 'Condition' as row, 'Sex' as column.
pivot_table = pd.pivot_table(melted_df,
                             index=['Condition'],
                             columns=['Sex'],
                             aggfunc='size',  # You can use 'count' if you prefer
                             fill_value=0)

In [59]:
#This adds a total column
pivot_table['Total'] = pivot_table.sum(axis=1)

In [60]:
#This rearranges from A-Z to total size descending.
pivot_table = pivot_table.sort_values(by='Total', ascending=False)

In [61]:
#Here is the end product
print(pivot_table)

Sex           Female  Male  Total
Condition                        
Diabetes          47    58    105
Anemia            52    51    103
Allergies         47    55    102
Migraine          52    50    102
Osteoporosis      45    57    102
Asthma            53    48    101
Depression        50    49     99
Hypertension      50    48     98
Obesity           45    52     97
Arthritis         48    43     91
